In [ ]:
import librosa
import parselmouth
import numpy as np
import numpy as np
from scipy.spatial.distance import pdist, squareform

def rpde2(data, m, tau, t):
    # Embed the data
    data = data[data != 0]
    embedded_data = np.array([data[i:i+m*tau:tau] for i in range(len(data) - (m-1)*tau - t + 1)])
    
    # Calculate the recurrence matrix
    dist = pdist(embedded_data, 'chebyshev')
    recurrence_matrix = squareform(dist < np.percentile(dist, 20))

    # Calculate RPDE
    epsilon = 1e-10  # Small constant to avoid taking log2(0)
    rpde = -np.sum(recurrence_matrix * np.log2(recurrence_matrix + epsilon)) / np.sum(recurrence_matrix)

    return rpde

def grassberger_procaccia(data, m, r):
    def _maxdist(x_i, x_j):
        return max([abs(ua - va) for ua, va in zip(x_i, x_j)])

    def _phi(m):
        x = [[data[j] for j in range(i, i + m - 1 + 1)] for i in range(N - m + 1)]
        C = [len([1 for x_j in x if _maxdist(x_i, x_j) <= r]) / (N - m + 1.0) for x_i in x]
        return (N - m + 1.0)**(-1) * sum(np.log(C))

    N = len(data)
    return abs(_phi(m+1) - _phi(m))

def extract_audio_features(fileName):
    
    features = {}

    # Load the audio file
    audio, sr = librosa.load(fileName)
    # Load the audio file
    snd = parselmouth.Sound(fileName)

    # Calculate the fundamental frequency
    f0, voiced_flag, voiced_probs = librosa.pyin(audio, fmin=75, fmax=600)
    valid_f0 = f0[~np.isnan(f0)]

    # Compute the average fundamental frequency
    average_f0 = valid_f0.mean()
    max_f0 = np.max(valid_f0)
    min_f0 = np.min(valid_f0)
    jitter_percent = (np.std(valid_f0) / np.mean(valid_f0)) * 100
    jitter_absolute = np.mean(np.abs(np.diff(valid_f0)))
    rap = np.mean(np.abs(np.diff(valid_f0)))
    ppq = np.mean(np.abs(np.diff(valid_f0, n=2)))
    
    
    features['average_f0'] = average_f0
    features['max_f0'] = max_f0
    features['min_f0'] = min_f0
    features['jitter_percent'] = jitter_percent
    features['jitter_absolute'] = jitter_absolute
    features['rap'] = rap
    features['ppq'] = ppq
    
    # Calculate the amplitude envelope
    amplitude_envelope = np.abs(librosa.effects.hpss(audio)[0])

    # Compute the time array for the amplitude envelope
    time = np.arange(len(amplitude_envelope)) / sr

    shimmer_apq3 = np.mean(np.abs(np.diff(amplitude_envelope, n=3)))
    shimmer_apq5 = np.mean(np.abs(np.diff(amplitude_envelope, n=5)))
    mdvp_shimmer = np.mean(np.abs(np.diff(amplitude_envelope)))
    mdvp_shimmer_db = 20 * np.log10(np.mean(np.abs(np.diff(amplitude_envelope))))
    jitter_ddp = np.mean(np.abs(np.diff(np.diff(amplitude_envelope))))
    mdvp_apq = np.mean(np.square(np.diff(amplitude_envelope, n=2)))
    shimmer_dda = np.mean(np.abs(np.diff(amplitude_envelope))) / (np.max(amplitude_envelope) - np.min(amplitude_envelope))
    harmonic_energy = np.mean(amplitude_envelope ** 2)
    noise_energy = np.mean((audio - amplitude_envelope) ** 2)
    nhr = noise_energy / harmonic_energy
    hnr = harmonic_energy / noise_energy
    
    features['shimmer_apq3'] = shimmer_apq3
    features['shimmer_apq5'] = shimmer_apq5
    features['mdvp_shimmer'] = mdvp_shimmer
    features['mdvp_shimmer_db'] = mdvp_shimmer_db
    features['jitter_ddp'] = jitter_ddp
    features['mdvp_apq'] = mdvp_apq
    features['shimmer_dda'] = shimmer_dda
    features['nhr'] = nhr
    features['hnr'] = hnr
    	
    # print("Average vocal fundamental frequency (MDVP:Fo):", average_f0, "Hz")
    # print("Maximum vocal fundamental frequency (MDVP:Fhi):", max_f0, "Hz")
    # print("Minimum vocal fundamental frequency (MDVP:Flo):", min_f0, "Hz")
    # Compute the MDVP:Jitter (Abs)
    # print("MDVP:Jitter (%):", jitter_percent)
    # print("MDVP:Jitter (Abs):", jitter_absolute)
    # Compute the MDVP:PPQ (Five measures of variation in fundamental frequency)
    # print("MDVP:RAP:", rap)
    # print("MDVP:PPQ:", ppq)
    
    # print("Jitter:DDP:", jitter_ddp)
    # print("MDVP:Shimmer:", mdvp_shimmer)
    # print("MDVP:Shimmer (dB):", mdvp_shimmer_db)
    # print("Shimmer:APQ3:", shimmer_apq3)
    # print("Shimmer:APQ5:", shimmer_apq5)
    # print("MDVP:APQ:", mdvp_apq)
    # print("Shimmer:DDA:", shimmer_dda)
    # print("NHR:", nhr)
    # print("HNR:", hnr)




    # Extract fundamental frequency (F0) contour
    pitch = snd.to_pitch()
    unit = "Hertz"  # or "Mel", "LogHertz", "ERB", "Semitones", "Bark", depending on your needs

    # Define time range
    start_time = 0.0  # start of the sound
    end_time = snd.get_total_duration()  # end of the sound

    # Use the defined time and quantile range
    spread1 = parselmouth.praat.call(pitch, "Get quantile", start_time, end_time, 0.75, unit) - parselmouth.praat.call(pitch, "Get quantile", start_time, end_time, 0.25, unit)
    spread2 = parselmouth.praat.call(pitch, "Get quantile", start_time, end_time, 0.90, unit) - parselmouth.praat.call(pitch, "Get quantile", start_time, end_time, 0.10, unit)


    # Convert Pitch object to PointProcess object
    pointProcess = parselmouth.praat.call(pitch, "To PointProcess")
    # Get the number of points
    num_points = parselmouth.praat.call(pointProcess, "Get number of points")
    # Get the intervals
    intervals = [parselmouth.praat.call(pointProcess, "Get time from index", i+1) - parselmouth.praat.call(pointProcess, "Get time from index", i) for i in range(1, num_points)]
    # Calculate the probabilities
    probabilities = np.bincount(intervals) / len(intervals)
    # Calculate the entropy
    ppe = -np.sum(probabilities * np.log2(probabilities))

    features['spread1'] = spread1
    features['spread2'] = spread2
    features['ppe'] = ppe

    # print("spread1:", spread1)
    # print("spread2:", spread2)
    # print("PPE (Pitch Period Entropy):", ppe)
    # Convert Pitch object to numpy array
    pitch_values = pitch.selected_array['frequency']

    # Calculate RPDE
    rpde = rpde2(pitch_values, m=2, tau=1, t=1)

    # Calculate D2
    d2 = grassberger_procaccia(pitch_values, m=2, r=0.2 * np.std(pitch_values))

    features['rpde'] = rpde
    features['d2'] = d2
    
    return features

headers = ['Person', 'Wav file', 'average_f0', 'max_f0', 'min_f0', 'jitter_percent', 'jitter_absolute', 'rap', 'ppq', 'shimmer_apq3', 'shimmer_apq5', 'mdvp_shimmer', 'mdvp_shimmer_db', 'jitter_ddp', 'mdvp_apq', 'shimmer_dda', 'nhr', 'hnr', 'spread1', 'spread2', 'ppe', 'rpde', 'd2']

In [ ]:
#Turn audio to CSV

import os
import pandas as pd


# Define the root directory containing the audio files
root_directory = "/home/pratik/Desktop/Plaksha_Docs/Semester_4/MLPR/Projects/Italian Parkinson's Voice and speech/28 People with Parkinson's disease"

# Initialize an empty list to store all extracted features
all_features = []

# Iterate over all subdirectories and WAV files
for person_folder in os.listdir(root_directory):
    person_folder_path = os.path.join(root_directory, person_folder)
    if os.path.isdir(person_folder_path):
        for wav_file in os.listdir(person_folder_path):
            if wav_file.endswith('.wav'):
                wav_file_path = os.path.join(person_folder_path, wav_file)
                # Extract features from the current WAV file
                features = extract_audio_features(wav_file_path)
                # Append the extracted features along with the filename and person's name
                all_features.append([person_folder, wav_file] + list(features.values()))

# Convert the list of features to a pandas DataFrame
df = pd.DataFrame(all_features, columns=headers)

# Write the DataFrame to a CSV file
csv_file_path = "28 People with Parkinson's disease.csv"
df.to_csv(csv_file_path, index=False)

print(f"Audio analysis results saved to {csv_file_path}")


In [ ]:
import os
import pandas as pd


# Define the root directory containing the audio files
root_directory = "/home/pratik/Desktop/Plaksha_Docs/Semester_4/MLPR/Projects/Italian Parkinson's Voice and speech/22 Elderly Healthy Control"

# Initialize an empty list to store all extracted features
all_features = []

# Iterate over all subdirectories and WAV files
for person_folder in os.listdir(root_directory):
    person_folder_path = os.path.join(root_directory, person_folder)
    if os.path.isdir(person_folder_path):
        for wav_file in os.listdir(person_folder_path):
            if wav_file.endswith('.wav'):
                wav_file_path = os.path.join(person_folder_path, wav_file)
                # Extract features from the current WAV file
                features = extract_audio_features(wav_file_path)
                # Append the extracted features along with the filename and person's name
                all_features.append([person_folder, wav_file] + list(features.values()))

# Convert the list of features to a pandas DataFrame
df = pd.DataFrame(all_features, columns=headers)

# Write the DataFrame to a CSV file
csv_file_path = 'audio_analysis_results.csv'
df.to_csv(csv_file_path, index=False)

print(f"Audio analysis results saved to {csv_file_path}")

In [1]:
#Normalize kaggle data
import pandas as pd

# Read the CSV file
df = pd.read_csv("parkinsons_fold.csv")

# Normalize numerical columns except 'Person' and 'Wav file'
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()) if x.name not in ['Person', 'Wav file', 'status', 'rep'] else x)

# Write the normalized data back to a new CSV file
df.to_csv('normalized_english_kaggle.csv', index=False)


In [ ]:
import pandas as pd

# Read the two CSV files
df1 = pd.read_csv('normalized_english_kaggle.csv')
df2 = pd.read_csv('combined_file_italian.csv')

# Ensure columns are in the same order
df2 = df2[df1.columns]

# Concatenate the two dataframes vertically
combined_df = pd.concat([df1, df2], ignore_index=True)

# Write the combined data to a new CSV file
combined_df.to_csv('final_Data.csv', index=False)